In [1]:
import pandas as pd
import os
import cv2
import numpy as np
import json

Year='1938'
Showa='13'

In [2]:
def Detect_Unit(Json,Dept,Office):
    NewList=Json['fields']
    Dict=list()
    for d in NewList:
        try:
            newDict={}
            newDict['inferText']=d['inferText']
            newDict['boundingPoly']=d['boundingPoly']
            Dict.append(newDict)
        except KeyError:
            continue
    
    res = [d
       for d in Dict 
       if ( d['inferText'][-1]=='掛')]

    if len(res)!=0:
        res = res[0]['boundingPoly']['vertices']
        Edge=max(int(d['x']) for d in res)
        return(Edge)
    else:
        return(None)

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

### CLOVA FUNCTION ###
import requests
import uuid
import time
import json
import cv2
import base64

api_url = 'https://deelieyxuc.apigw.ntruss.com/custom/v1/1972/ebd01bcbf693d069817622e9839e20914143c7d0d8953eddee40e8b0af96c95b/general'
secret_key = 'S1NmVXpYZlJ0cGJ0ZEFRZXVlbkRkaHFReE9FcHNTQ0U='

def Clova(Year,Office,Num):
    path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Raw_Data\\Office_Level\\"+Year+"\\"+Dept+"\\"+Office+"\\"
    with open(path+"Image"+str(Num)+".png",'rb') as f:
         file_data = f.read()

    request_json = {
            'images': [
                {
                    'format': 'jpg',
                    'name': 'demo',
                    'data':base64.b64encode(file_data).decode()}],
            'requestId': str(uuid.uuid4()),
            'version': 'V2',
            'timestamp': int(round(time.time() * 1000)),
            'lang':'ja'
            }
    payload = json.dumps(request_json).encode("UTF-8")
    headers = {'X-OCR-SECRET': secret_key,
              'Content-Type': 'application/json'}
    response = requests.request("POST", api_url, headers=headers, data = payload)
    Json=json.loads(response.text)['images'][0]
    
    return Json

def Clova2(Year,Dept,Office,Num,LeftEdge,RightEdge):
    path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Raw_Data\\Office_Level\\"+Year+"\\"+Dept+"\\"+Office+"\\"
    stream = open(path+"Image"+str(Num)+'.png', "rb")
    bytes = bytearray(stream.read())
    numpyarray = np.asarray(bytes, dtype=np.uint8)
    img = cv2.imdecode(numpyarray, cv2.IMREAD_UNCHANGED)
    HH,WW=img.shape[:2]
    cropped=img[0:HH,LeftEdge:RightEdge]
    
    temp_path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Raw_Data\\Temp\\"
    cv2.imwrite(temp_path+"Temp.jpg",cropped)
    with open(temp_path+"Temp.jpg",'rb') as f:
        file_data = f.read()
    
    request_json = {
            'images': [
                {
                    'format': 'jpg',
                    'name': 'demo',
                    'data':base64.b64encode(file_data).decode()}],
            'requestId': str(uuid.uuid4()),
            'version': 'V2',
            'timestamp': int(round(time.time() * 1000)),
            'lang':'ja'
            }
    payload = json.dumps(request_json).encode("UTF-8")
    headers = {'X-OCR-SECRET': secret_key,
              'Content-Type': 'application/json'}
    response = requests.request("POST", api_url, headers=headers, data = payload)
    Json=json.loads(response.text)['images'][0]
    
    return Json

In [3]:
#Extract Location of units.
def ExtractUnit(Year,Dept,Office):
    Dict=list()
    PageRange=dta[Year][Dept][Office]['Page_Range']
    NumberList=list(range(PageRange[0]-PageRange[0]+1,PageRange[-1]-PageRange[0]+1+1))
    
    #Apply OCR and extract columns with unit names
    for n in NumberList: 
        newDict=list()
        Json=Clova(Year,Office,n)

        NewList=Json['fields']
        for d in NewList:
            try:
                newnewDict={}
                newnewDict['inferText']=d['inferText']
                newnewDict['boundingPoly']=d['boundingPoly']
                newnewDict['Page']=int(n)
                newDict.append(newnewDict)
            except KeyError:
                continue

        res = [d['boundingPoly']
           for d in newDict 
           if d['inferText'][-1]=='掛']

        for col in res:
            col2=col['vertices']
            EdgeL=min(d['x'] for d in col2)
            EdgeR=max(d['x'] for d in col2)
            col['EdgeL']=int(EdgeL)
            col['EdgeR']=int(EdgeR+5)
            col['Page']=int(n)
            col['vertices']=None
        Dict.append(res)
    
    #Reformat OCR-ed Image into dictionary
    Count=0
    for page in Dict:
        if page==[]:
            continue
        Page=page[0]['Page']
        for unit in page:
            try:
                LeftEdge,RightEdge=unit['EdgeL'],unit['EdgeR']
                Json=Clova2(Year,Dept,Office,Page,LeftEdge,RightEdge)
                for element in Json['fields']:
                    res = element['boundingPoly']['vertices']
                    TopY=max(int(d['y']) for d in res)
                    BtmY=min(int(d['y']) for d in res)
                    element['YCenter']=(TopY+BtmY)//2

                NameList=sorted(Json['fields'], key=lambda d: d['YCenter']) 
                Text=''.join([element['inferText'] for element in NameList])
                unit['Text']=Text.split('掛')[0]+'掛'
            except:
                continue
        newpage={x['Text']:{'Page':x['Page'],'StartLocation':x['EdgeR']} for x in page}
        Dict[Count]=newpage
        Count=Count+1
    
    #Concatate into single nested dictionary
    for page in Dict:    
        Dict[0].update(page)
    
    return (Dict,Count)

In [18]:
def Organize(Dict):
    UnitList=list(Dict[0].keys())
    PageList=[]
    for Unit in UnitList:
        Page=Dict[0][Unit]['Page']
        PageList.append(Page)
    XEnd=0.0
    EndPage=max(PageList)

    Frame=pd.DataFrame.from_dict(Dict[0],orient='index')
    Frame=Frame.sort_values(by = ['Page', 'StartLocation'], ascending = [True, False])
    Frame['EndPage']=Frame['Page'].shift(periods=-1)[:-1].astype(int)
    Frame['EndLocation']=Frame['StartLocation'].shift(periods=-1)[:-1].astype(int)
    Frame['EndPage'][-1]=float(EndPage)
    Frame['EndPage']=Frame['EndPage'].astype(int)
    Frame['EndLocation'][-1]=float(XEnd)
    Frame['EndLocation']=Frame['EndLocation'].astype(int)
    Frame['Page_Range']=Frame.apply(lambda x : list(range(x['Page'],x['EndPage']+1)),1)
    
    Frame.index = Frame.index.map(str)
    Frame.columns = Frame.columns.map(str)
    js = str(Frame.to_dict(orient='index')).replace("'", '"')

    return(js)

In [19]:
path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Raw_Data\\Splited\\"+Year+"\\"
os.chdir(path)
df = pd.read_csv(r'C:/Users/Keitaro Ninomiya/Box/Research Notes (keitaro2@illinois.edu)/Tokyo_Jobs/Processed_Data/Index/S'+Showa+'.csv')
df=df.drop(df.columns[0], axis=1)

file_path='C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Processed_Data\\'+Year+'\\DataFrame.json'
with open(file_path, encoding="utf-8") as f:
    dta = json.loads(f.read())

In [20]:
FailedList=[]
for n in range(0,len(df)):
    Row  = df.iloc[n]
    NxRow= df.iloc[n+1]

    ###Collect Location information###
    Dept=Row["Dept"]
    NxDept=NxRow["Dept"]

    Office=Row["Office"]
    NxOffice=NxRow["Office"]

    #Get info of units
    try:        
        #Get info of units
        Dict=ExtractUnit(Year,Dept,Office)
        Dict=json.loads(Organize(Dict[0]))
        #Add info to dataframe
        dta[Year][Dept][Office]['Units']=Dict
        dta[Year][Dept][Office]['Units']['List']=list(Dict.keys())
    except:
        print('Error at '+Dept+" "+Office)
        FailedList.append(list((Dept,Office)))
        continue


C:\Temp\ipykernel_9996\4222201218.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Frame['EndPage'][-1]=float(EndPage)
C:\Temp\ipykernel_9996\4222201218.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Frame['EndLocation'][-1]=float(XEnd)
C:\Temp\ipykernel_9996\4222201218.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Frame['EndPage'][-1]=float(EndPage)
C:\Temp\ipykernel_9996\4222201218.py:16: SettingWithCopyWarning: 
A v

Error at 企画局 企画課


C:\Temp\ipykernel_9996\4222201218.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Frame['EndPage'][-1]=float(EndPage)
C:\Temp\ipykernel_9996\4222201218.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Frame['EndLocation'][-1]=float(XEnd)


Error at 企画局 統計課
Error at 企画局 都市計画課


C:\Temp\ipykernel_9996\4222201218.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Frame['EndPage'][-1]=float(EndPage)
C:\Temp\ipykernel_9996\4222201218.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Frame['EndLocation'][-1]=float(XEnd)
C:\Temp\ipykernel_9996\4222201218.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Frame['EndPage'][-1]=float(EndPage)
C:\Temp\ipykernel_9996\4222201218.py:16: SettingWithCopyWarning: 
A v

...

Error at 電気研究所 研究課


C:\Temp\ipykernel_9996\4222201218.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Frame['EndPage'][-1]=float(EndPage)
C:\Temp\ipykernel_9996\4222201218.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Frame['EndLocation'][-1]=float(XEnd)


Error at 臨時建築部 管理課
Error at 臨時建築部 第一工菅課
Error at 臨時建築部 第二工菅課


C:\Temp\ipykernel_9996\4222201218.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Frame['EndPage'][-1]=float(EndPage)
C:\Temp\ipykernel_9996\4222201218.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Frame['EndLocation'][-1]=float(XEnd)


IndexError: single positional indexer is out-of-bounds

In [21]:
FailRate=len(FailedList)/len(df)
if len(FailedList)/len(df)<0.2:
    print('Fantastic!! Success Rate is '+str(1-(len(FailedList)/len(df))))
else:
    print('残念...Success Rate is '+str(1-(len(FailedList)/len(df))))
DF=pd.read_csv('C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Processed_Data\\Records.csv')
DF.loc[int(Year)-1934, 'Unit'] = 1-FailRate
DF.to_csv('C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Processed_Data\\Records.csv')

残念...Success Rate is 0.4642857142857143


In [23]:
json_object = json.dumps(dta, indent=4,
                        cls=NpEncoder)
save_path='C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Processed_Data\\'+str(Year)+'\\'
with open(save_path+"DataFrame.json", "w") as outfile:
    outfile.write(json_object)